### ※注意事項
- このノートブックはTellus開発環境にアップロードして利用してください
- 学習を行った `golf-model.pth` ファイルもアップロードして、ノートと同じフォルダ内に保存してください

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

import torch.nn as nn
import torch
from torch.autograd import Variable

import torchvision.transforms as transforms
import torchvision.models as models

%matplotlib inline

In [ ]:
TOKEN = 'YOUR_TOKEN'
def get_image(rspid,productid,z,x,y):
    url = f"https://gisapi.tellusxdp.com/blend/av2ori/{rspid}/{productid}/{z}/{x}/{y}.png"
    headers = {
        'Authorization': 'Bearer ' + TOKEN
    }
    r = requests.get(url, headers=headers)
    return Image.open(BytesIO(r.content))

In [ ]:
im_array = get_image('D069P3', 'ALAV2A054262870', 14, 14529, 6441)
plt.imshow(im_array)
im_array.save('positive.png')

In [ ]:
im_array = get_image('D069P3', 'ALAV2A054262870', 14, 14528, 6441)
plt.imshow(im_array)
im_array.save('negative.png')

In [ ]:
model = models.resnet18(pretrained=True)

In [ ]:
model.fc = nn.Linear(512, 2)

In [ ]:
model.load_state_dict(torch.load('./golf-model.pth', map_location=torch.device('cpu')))

In [ ]:
model.eval()

imsize = 256
loader = transforms.Compose([transforms.Scale(imsize), transforms.ToTensor()])

def image_loader(image_name):
    image = Image.open(image_name).convert("RGB")
    image = loader(image)
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)
    return image

m = nn.Softmax(dim=1)

In [ ]:
image = image_loader('./positive.png')
m(model(image))

In [ ]:
image = image_loader('./negative.png')
m(model(image))